In [1]:
import os

In [ ]:
path = 'C:/Users/jpjaldin/Desktop/QUEMA/COMPROBANTES'
documents = os.listdir(path)

In [ ]:
documents

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

gauth = GoogleAuth()
gauth.LocalWebserverAuth() # client_secrets.json need to be in the same directory as the script
drive = GoogleDrive(gauth)

In [ ]:
# recibe un FILENAME y crea un diccionario {COD; FILENAME}
def convert_to_dict(file):
    dic = {}
    dic['cod'] = file.split('_')[0] + '_' + file.split('_')[2]
    dic['filename'] = file
    return dic

In [ ]:
# convertir la lista de FILENAMES a diccionarios de COD mas FILENAME
list_of_dicts = list(map(convert_to_dict, documents))

In [ ]:
list_of_dicts

In [ ]:
# subir los archivos a DRIVE y obtener los links
dic_urls = {}
# View all folders and file in your Google Drive
fileList = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file in fileList:
  #print('Title: %s, ID: %s' % (file['title'], file['id']))
  # Get the folder ID that you want
  if(file['title'] == "CANA_QUEMADA_2021"):
      fileID = file['id']

for file in list_of_dicts:
    # para .PDF
    file1 = drive.CreateFile({"mimeType": "application/pdf", "parents": [{"kind": "drive#fileLink", "id": fileID}]})
    # para .DOCX
    #file1 = drive.CreateFile({"mimeType": "application/vnd.openxmlformats-officedocument.wordprocessingml.document", "parents": [{"kind": "drive#fileLink", "id": fileID}]})
    file1['title'] = file['filename']
    file1.SetContentFile(path + '/' + file['filename'])
    file1.Upload() # Upload the file.
    print('Created file %s with mimeType %s' % (file1['title'], file1['mimeType']))
    permission = file1.InsertPermission({
                        'type': 'anyone',
                        'value': 'anyone',
                        'role': 'reader'})
    dic_urls[file['cod']] = file1['alternateLink']
dic_urls
print('end')

In [ ]:
dic_urls

In [ ]:
for key in dic_urls:
    query = {'query': 'update dataset_269481 set link_comprobante = \'{link}\' where split_part(canhero, \' / \', 1) || \'_\' || to_char(fecha_inspeccion, \'dd-mm-yyyy\')=\'{cod}\''.format(link=dic_urls[key], cod=key)}
    project = 'https://app.amigocloud.com/api/v1/projects/30545/sql'
    res = amigocloud.post(project, query)
    print(key)
print('end')